In [ ]:
from agera5tools.wdp import AgERA5WeatherDataProvider
import pickle

In [ ]:
# # Import weather station coordinates 
# WA_OR = pd.read_csv('../data/RWS_wa_or_coord.csv')
# WA_OR is where the value below from 

# wdp = AgERA5WeatherDataProvider(longitude=-119.58, latitude=46.58)
# the location coornidates are the experimental site
## run once and save to pkl then load the pkl afterwards
# wdp = AgERA5WeatherDataProvider(longitude=73.13, latitude=23.84)
# with open('../intermediate_data/wdp_ind.pickle', 'wb') as f:
#     # Pickle the 'data' dictionary using the highest protocol available.
#     pickle.dump(wdp, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../intermediate_data/wdp_ind.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    wdp = pickle.load(f)

In [ ]:
%matplotlib inline
import os, sys
data_dir = os.path.join(os.getcwd(), "data")
from itertools import product
import yaml
import pandas as pd
import pcse
from pcse.models import Wofost72_WLP_FD, Wofost72_PP
from pcse.base import ParameterProvider
from pcse.exceptions import WeatherDataProviderError
import matplotlib.pyplot as plt


print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

# setting font and size 
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}
# change font
plt.rc('font', **font)

In [ ]:
# load two data import functions from the pcse.fileinput class
from pcse.fileinput import YAMLCropDataProvider, CABOFileReader
# load the available crop list  
cropd = YAMLCropDataProvider(fpath='../data',
                              force_reload=True)  
# load the soil information
soild = CABOFileReader('../data/ec3 sandyloam.soil')
# load one data import function from the pcse.util class
from pcse.util import WOFOST72SiteDataProvider
# define the site initial condiations 
# WAV is the initial soil water content = 0; CO2 is the level of CO2 in the atmosphere = 360 ppm
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360)
print(sited)
# help(WOFOST72SiteDataProvider)


In [ ]:
StartDate = [str(i) + '-11-01' for i in range(2013, 2021)] 
EndDate = [str(i+1) + '-02-28' for i in range(2013, 2022)] 

In [ ]:
# Loop over crops, soils and years
agropotato = """
- {StartDate}:
    CropCalendar:
        crop_name: potato
        variety_name: {Cultivar}
        crop_start_date: {StartDate}
        crop_start_type: sowing
        crop_end_date: {EndDate}
        crop_end_type: maturity
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""
cultivars = [
    "Fontane", 
    "Markies",
    "Premiere",
    "Festien", 
    "Innovator"]

res = []
# for i 
for i, inputs in enumerate(zip(StartDate, EndDate)):
    # print(i)
    # print(inputs)
    std, etd = inputs
    for c in cultivars:
        run_id = "{StartDate}_{EndDate}".format(StartDate=std, EndDate = etd)
        # print(run_id)
        agromanagement = yaml.safe_load(agropotato.format(StartDate=std, EndDate = etd, Cultivar = c))
        cropd.set_active_crop('potato', c)
        parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)
        # increase the leave span
        for span in [35, 40, 45]:
            parameters.set_override("SPAN", span)
            wofost = Wofost72_PP(parameters, wdp, agromanagement)
            wofost.run_till_terminate()
            output = wofost.get_output()
            df = pd.DataFrame(output).set_index("day")
            df['Run_id'] = run_id
            df['Cultivar'] = c
            df['SPAN'] = span
             # append each potential yield simulation results for each cultivar
            res.append(df)
            print('Cultivar' + c + run_id  + 'SPAN ' + str(span))


In [ ]:
df_res = pd.concat(res)

In [ ]:
span_35 = df_res[df_res['SPAN'] == 35]
span_40 = df_res[df_res['SPAN'] == 40]
span_45 = df_res[df_res['SPAN'] == 45]

In [ ]:
df_res = pd.concat(res)
colors = ['k','r','g','b', 'm']
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
for c, cultivar in zip(colors, cultivars):
    for var, axe in zip(["DVS", "TAGP", "LAI", "TWSO","SM"], axes.flatten()):
        span_35[span_35['Cultivar']==cultivar][var].plot(ax=axe, color=c, label = cultivar)
        # plt.scatter(actual_yield['HarvestDate'].iloc[0:3], actual_yield['Yield_t_ha'][0:3]*1000*0.2, color = 'r')
        axe.set_title(var)  
# Modify the legend position - plt.legend will put legends at the last plot, 
# fig.legend contains all duplicated legend names for each simulation set 
# axes is the good one 
axes[0][0].legend( loc='upper left')
fig.autofmt_xdate()
plt.suptitle("span_35")
fig.savefig("playingSPAN_35_IND.png")


In [ ]:
colors = ['k','r','g','b', 'm']
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
for c, cultivar in zip(colors, cultivars):
    for var, axe in zip(["DVS", "TAGP", "LAI", "TWSO","SM"], axes.flatten()):
        span_40[span_40['Cultivar']==cultivar][var].plot(ax=axe, color=c, label = cultivar)
        # plt.scatter(actual_yield['HarvestDate'].iloc[0:3], actual_yield['Yield_t_ha'][0:3]*1000*0.2, color = 'r')
        axe.set_title(var)  
# Modify the legend position - plt.legend will put legends at the last plot, 
# fig.legend contains all duplicated legend names for each simulation set 
# axes is the good one 
axes[0][0].legend( loc='upper left')
fig.autofmt_xdate()
plt.suptitle("span_40")
fig.savefig("playingSPAN_40_IND.png")


In [ ]:
colors = ['k','r','g','b', 'm']
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,10))
for c, cultivar in zip(colors, cultivars):
    for var, axe in zip(["DVS", "TAGP", "LAI", "TWSO","SM"], axes.flatten()):
        span_45[span_45['Cultivar']==cultivar][var].plot(ax=axe, color=c, label = cultivar)
        # plt.scatter(actual_yield['HarvestDate'].iloc[0:3], actual_yield['Yield_t_ha'][0:3]*1000*0.2, color = 'r')
        axe.set_title(var)  
# Modify the legend position - plt.legend will put legends at the last plot, 
# fig.legend contains all duplicated legend names for each simulation set 
# axes is the good one 
axes[0][0].legend( loc='upper left')
fig.autofmt_xdate()
plt.suptitle("span_45")
fig.savefig("playingSPAN_45_IND.png")


In [ ]:
# effect size 
df_fontane = df_res[df_res['Cultivar'] == 'Fontane'].loc[:, ['LAI','SPAN']]
df_fontane_wide = df_fontane.pivot(columns = 'SPAN', values = "LAI")
df_fontane_wide

In [ ]:
df_fontane_wide.columns

In [ ]:
df_fontane_wide['changes_10'] = df_fontane_wide[45] - df_fontane_wide[35] 
df_fontane_wide['changes_5'] = df_fontane_wide[40] - df_fontane_wide[35] 

In [ ]:
df_fontane_wide.loc[:,['changes_10','changes_5']].plot()


In [ ]:
df_fontane_wide.columns 

In [ ]:
# what happened at year 2019 and 2022
weather =pd.DataFrame(  wdp.export())
weather.DAY = pd.to_datetime(weather.DAY)

In [ ]:
weather.set_index('DAY', inplace = True)

In [ ]:
weather['TMAX'].plot()
weather['TMIN'].plot()


### Next step 
1. choose a cultivar - which one does not matter I think since we are not after the accuracy but the variation over the seasons. 
2. demonstrate the relationship between parameter values and output? LAI or twso? 

$$effect size = \delta{Y}/\delta{P}$$